## Need to get accurate detection working.

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import pandas as pd

class sizeme():
    """ Class to change html fontsize of object's representation"""
    def __init__(self,ob, size=50, height=120):
        self.ob = ob
        self.size = size
        self.height = height
    def _repr_html_(self):
        repl_tuple = (self.size, self.height, self.ob._repr_html_())
        return u'<span style="font-size:{0}%; line-height:{1}%">{2}</span>'.format(*repl_tuple)

pd.options.display.max_columns = 9999
pd.set_option('display.width', 9999)

Use the code from `diffimTests.py`.

In [60]:
import diffimTests as dit
reload(dit)

# Let's try w same parameters as ZOGY paper.
sky = 300.

testObj = dit.DiffimTest(imSize=(512,512), sky=sky, offset=[0,0], psf_yvary_factor=0., 
                         varSourceChange=[1500., 1600., 1800., 2000., 2200., 2400., 2600., 2800.],
                         theta1=0., theta2=-45., im2background=0., n_sources=50, sourceFluxRange=(500,30000), 
                         seed=66, psfSize=13)

print dit.computeClippedImageStats(testObj.im1.im)
print dit.computeClippedImageStats(testObj.im2.im)
print dit.computeClippedImageStats(testObj.im1.var)
print dit.computeClippedImageStats(testObj.im2.var)

Template PSF: [1.6, 1.6] 0.0
Science PSF: [1.8, 2.2] -45.0
0.824621125124
Offset: [0, 0]
PSF y spatial-variation: 0.0 0.0
Variable source: 37 245.899250131 279.361590046 1423.89964966 2923.89964966
Variable source: 2 287.153594381 287.464482497 11199.2212522 12799.2212522
Variable source: 18 262.668862066 301.46927489 20097.6617321 21897.6617321
Variable source: 15 305.718449906 258.588102301 27152.9264726 29152.9264726
Variable source: 23 286.863456899 311.26461378 26439.9102459 28639.9102459
Variable source: 7 193.358717505 245.822023119 16949.7483921 19349.7483921
Variable source: 45 293.525936039 203.144579256 4226.31645367 6826.31645367
Variable source: 33 220.821923649 196.83656518 14224.0855888 17024.0855888
(0.058941207521673616, 17.142582277831181, -74.0, 1741.5550996425638)
(0.16640941424314851, 17.212254472642801, -82.0, 1169.0572762553779)
(300.05894120752163, 17.142582277831185, 226.0, 2041.5550996425638)
(300.16640941424311, 17.212254472642801, 218.0, 1469.0572762553779)


In [61]:
src = testObj.runTest(returnSources=True)

In [62]:
changedCentroid = np.array(testObj.centroids[testObj.changedCentroidInd, :])
print changedCentroid

print src['AL'][['base_NaiveCentroid_x', 'base_NaiveCentroid_y', 'base_PsfFlux_fluxSigma', 'base_PsfFlux_flag']]
print src['ZOGY'][['base_NaiveCentroid_x', 'base_NaiveCentroid_y', 'base_PsfFlux_fluxSigma', 'base_PsfFlux_flag']]
# For SZOGY, the correct flux measurement is PeakLikelihoodFlux
print src['SZOGY'][['base_NaiveCentroid_x', 'base_NaiveCentroid_y', 'base_PeakLikelihoodFlux_fluxSigma', 'base_PsfFlux_flag']]

[[   245.89925013    279.36159005   1423.89964966   2923.89964966]
 [   287.15359438    287.4644825   11199.22125222  12799.22125222]
 [   262.66886207    301.46927489  20097.66173206  21897.66173206]
 [   305.71844991    258.5881023   27152.92647256  29152.92647256]
 [   286.8634569     311.26461378  26439.91024586  28639.91024586]
 [   193.35871751    245.82202312  16949.74839209  19349.74839209]
 [   293.52593604    203.14457926   4226.31645367   6826.31645367]
 [   220.82192365    196.83656518  14224.08558878  17024.08558878]]
   base_NaiveCentroid_x  base_NaiveCentroid_y  base_PsfFlux_fluxSigma base_PsfFlux_flag
2            220.281251            196.981653               55.819528             False
3            293.161952            203.164991               44.627419             False
4            193.111072            246.116698               57.651681             False
5            305.606008            258.429174               66.764066             False
6            245.311693

In [63]:
dist = np.sqrt(np.add.outer(src['AL'].base_NaiveCentroid_x, -changedCentroid[:, 0])**2. + \
               np.add.outer(src['AL'].base_NaiveCentroid_y, -changedCentroid[:, 1])**2.) # in pixels
print dist
matches = np.where(dist <= 1.5)
print matches
true_pos = len(matches[0])
false_neg = changedCentroid.shape[0] - len(matches[0])
false_pos = src['AL'].shape[0] - len(matches[0])
print true_pos, false_neg, false_pos

[[  86.27129218  112.51245572  112.75802692  105.33218657  132.26407373
    55.76920765   73.50350712    0.55980141]
 [  89.66428865   84.51333985  102.92502483   56.82769488  108.28296019
   108.53712704    0.36455619   72.6163111 ]
 [  62.38441002  102.7308879    88.89428524  113.29588484  114.16549669
     0.38491776  109.22338553   56.53691539]
 [  63.26976387   34.40262698   60.79513832    0.19468306   56.06127876
   112.9530633    56.58899634  104.79499222]
 [   1.23007945   42.83786577   28.96500722   63.53568224   53.05161529
    61.25921085   89.27531157   85.04663371]
 [  41.37089973    1.06530979   27.15679004   35.34382713   23.13250012
   102.1471471    85.29792777  112.37042659]
 [  27.25813503   28.38835371    0.50375892   60.81270374   26.58293607
    88.3811256   102.8637413   112.24852294]]
(array([0, 1, 2, 3, 4, 5, 6]), array([7, 6, 5, 3, 0, 1, 2]))
7 1 0


In [64]:
dist = np.sqrt(np.add.outer(src['SZOGY'].base_NaiveCentroid_x, -changedCentroid[:, 0])**2. + \
               np.add.outer(src['SZOGY'].base_NaiveCentroid_y, -changedCentroid[:, 1])**2.) # in pixels
print dist
matches = np.where(dist <= 1.5)
true_pos = len(matches[0])
false_neg = changedCentroid.shape[0] - len(matches[0])
false_pos = src['SZOGY'].shape[0] - len(matches[0])
print true_pos, false_neg, false_pos

[[  87.1614004   113.32639617  113.6381244   105.99553816  133.11320735
    56.44876739   73.76625309    1.0290128 ]
 [  89.72609575   84.63301002  103.01996376   56.9697138   108.40383698
   108.50752351    0.46240337   72.51956138]
 [  63.01859887  103.22988262   89.61686402  113.49020423  114.83512621
     0.83279673  108.87667845   55.64886005]
 [  62.89471505   34.41204922   60.66913923    0.79785649   56.20541253
   112.4034766    56.15556602  104.1820237 ]
 [   0.36793809   42.00708849   27.96577873   63.107761     52.05484481
    62.23477061   89.51881442   85.94558632]
 [  41.8352027     0.42471634   28.29405447   34.06419507   24.20371769
   102.33806184   84.16865896  111.90476193]
 [  27.07271371   28.51716987    0.71057056   60.89112055   26.79518961
    88.17580008  102.8405755   112.09670121]
 [  51.17514688   23.61167225   25.29790168   56.01825006    0.81024711
   113.36872644  108.16370805  131.54449663]
 [ 222.58393332  180.5509547   203.10409714  170.10780572  178.3

In [68]:
reload(dit)

# Let's try w same parameters as ZOGY paper.
sky = 300.

testObj = dit.DiffimTest(imSize=(512,512), sky=sky, offset=[0,0], psf_yvary_factor=0., 
                         varSourceChange=[1500., 1600., 1800., 2000., 2200., 2400., 2600., 2800.],
                         theta1=0., theta2=-45., im2background=0., n_sources=50, sourceFluxRange=(500,30000), 
                         seed=66, psfSize=13)

det = testObj.runTest()
print det

{'ZOGY': {'FP': 0, 'FN': 0, 'TP': 8}, 'SZOGY': {'FP': 1, 'FN': 0, 'TP': 8}, 'AL': {'FP': 0, 'FN': 1, 'TP': 7}}


Try some real tests!

In [ ]:
# Default 10 sources with same flux
def runTest(flux, seed=66, sky=300., n_sources=50):
    sky = 300.
    testObj = dit.DiffimTest(imSize=(512,512), sky=sky, offset=[0,0], psf_yvary_factor=0., 
                             varSourceChange=[flux, flux, flux, flux, flux, flux, flux, flux, flux, flux],
                             theta1=0., theta2=-45., im2background=0., n_sources=n_sources, 
                             sourceFluxRange=(500,30000), 
                             seed=seed, psfSize=13)
    det = testObj.runTest()
    return det


testResults = [runTest(f, seed) for f in [1500, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000] for \
              seed in [66, 67, 68, 69, 70]]
testResults

In [73]:
AL_FN = np.array([tr['AL']['FP'] for tr in testResults])
AL_FN

array([0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 4, 0, 1, 0, 0, 5, 0, 1, 1, 0, 5, 0,
       2, 2, 0, 5, 1, 2, 4, 0, 5, 0, 2, 5, 1, 5, 0, 3, 6, 2, 5, 0, 3, 7])